# Setup

## Imports

In [22]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path 
current_path = Path().resolve()
abs_path = str(current_path.parent)
sys.path.append(abs_path)

RAW_PATH = current_path.parent / 'data' / 'raw'
OUTPUT_PATH = current_path.parent / 'data' / 'output'

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.offline as offline
offline.init_notebook_mode(connected=True)

from h3 import h3
import folium
from pyathena import connect

EXTERNAL_LOCATION = 's3://athena-fgv/'

ModuleNotFoundError: No module named 'h3'

In [ ]:
!pip3.7 install h3

## Useful functions

## Connect With Athena

In [ ]:
from pyathena import connect

conn = connect(s3_staging_dir='s3://athena-fgv/stagging',
               region_name='us-east-2')

In [ ]:
def zip_columns(res):
    return list(zip(map(lambda x: x[0], res.description), res.fetchall()[0]))

def execute(query, conn, get_data=True):
    cursor = conn.cursor()
    res = cursor.execute(query)
    if get_data:
        data = zip_columns(res)
        cursor.close()
        return data
    else:
        cursor.close()

## Declare Variables

In [ ]:
# {'max_lon': -119.949377,
#  'max_lat': 39.716694,
#  'min_lon': -121.802217,
#  'min_lat': 38.246918}
# boundaries = ''

oms_region_table_name = 'osm.california_district_3'

# Get OSM slice of region

## Get region boundaries 

In [ ]:
query = """SELECT 
    max(longitude) max_lon,
    max(latitude) max_lat,
    min(longitude) min_lon,
    min(latitude) min_lat
FROM pems.pems_5_minute_metadata"""

In [ ]:
data = execute(query, conn)

In [ ]:
boundaries = dict(data)

## Create OSM table of region

In [93]:
query = f"""
CREATE TABLE {oms_region_table_name}
WITH (
      external_location = '{EXTERNAL_LOCATION}osm/california_district_3/3',
      format = 'ORC')
AS SELECT *
FROM osm.planet
WHERE lat < {boundaries['max_lat']}
  AND lat > {boundaries['min_lat']}
  AND lon < {boundaries['max_lon']}
  AND lon > {boundaries['min_lon']}
  """

In [88]:
execute(query, conn, get_data=False)

UnboundLocalError: local variable 'data' referenced before assignment

# Different try

## Get H3 hexagons that are in the boundary

In [259]:
def boundaries_to_geojson(boundaries):
    
    return {'type': 'Polygon',
 'coordinates': [[[boundaries['max_lat'], boundaries['max_lon']],
                  [boundaries['max_lat'], boundaries['min_lon']],
                  [boundaries['min_lat'], boundaries['min_lon']],
                  [boundaries['min_lat'], boundaries['max_lon']]]] }
    

In [289]:
def hex_coor_to_athena_geo(hex_coor):
    hex_coor = ''.join([str(h).replace(',', '').replace('[', '').replace(']', ',') for h in hex_coor])
    return 'POLYGON(('+ hex_coor[:-2] + '))'

In [290]:
def geojson_to_h3(geojson, res):
    
    return  list(map(lambda x: {'id': x,
                                'res': res,
                                'latitude': h3.h3_to_geo(x)[0],
                                'longitude': h3.h3_to_geo(x)[1],
                                'boundary': h3.h3_to_geo_boundary(x),
                                'boundary_athena': hex_coor_to_athena_geo(h3.h3_to_geo_boundary(x))}, h3.polyfill(geojson, res)))

In [291]:
geojson = boundaries_to_geojson(boundaries)
hexagons_in_region = geojson_to_h3(geojson, 8)

## Write to S3 as parquet

In [310]:
s3_url = 's3://fgv-bd/h3/res=8/calif_3.parquet.gzip'
df = pd.DataFrame(hexagons_in_region).set_index('res')
df.to_parquet(s3_url, compression='gzip')

## Join H3 with OSM

In [147]:
%timeit h3.geo_to_h3(38.50940595263018, -120.3511113181587, 8)

19.4 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [149]:
osm = pd.read_sql_query("""SELECT id, lat, lon FROM osm.california_district_3 LIMIT 100""", conn)

In [150]:
osm.head()

,id,lat,lon
0,1030979464,39.636424,-120.012501
1,1030979570,39.626327,-120.015171
2,1030979605,39.634838,-119.963224
3,1030979617,39.684711,-119.992431
4,1030979762,39.669065,-120.002910


In [154]:
%timeit osm.apply(lambda x: h3.geo_to_h3(x['lat'], x['lon'], 8), axis=1)

18.2 ms ± 1.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [159]:
osm = osm.to_dict(orient='record')

In [161]:
%timeit [h3.geo_to_h3(x['lat'], x['lon'], 8) for x in osm]

2.54 ms ± 266 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Different Try

Get just hexagons around the sensors


## Get sensors location

In [167]:
query = """
SELECT 
id, MAX(latitude) latitude, MAX(longitude) longitude
FROM pems.pems_station_metadata
GROUP BY id
"""

In [168]:
sensors = pd.read_sql_query(query, conn)

## Get hexagons around sensors

In [169]:
sensors['hexagon_8'] = sensors.apply(lambda x: h3.geo_to_h3(x['latitude'], x['longitude'], 8), axis=1)

In [186]:
len(h3.k_ring('88283284d1fffff', 2))

19

In [191]:
for i in range(7):
    sensors[f'neigh_{i}'] = sensors.apply(lambda x: h3.k_ring(x['hexagon_8'], i), 1)


In [194]:
for i in range(7):
    print('ring: ', i, 'size: ', len(h3.k_ring('88283284d1fffff', i)) * len(sensors))

ring:  0 size:  1341
ring:  1 size:  9387
ring:  2 size:  25479
ring:  3 size:  49617
ring:  4 size:  81801
ring:  5 size:  122031
ring:  6 size:  170307


In [ ]:
## 

In [201]:
sensors['hexagon_8'][0]

'882832b369fffff'

In [195]:
import folium

In [205]:

address = sensors['hexagon_8'][i]
latitude, longitude = sensors['latitude'][i], sensors['longitude'][i]

m = folium.Map(
    location=[latitude, longitude],
    zoom_start=13
)


folium.Polygon(
    h3.h3_to_geo_boundary(sensors['hexagon_8'][0]),
    tooltip=i
).add_to(m)

m

In [247]:
boundari = [h3.h3_to_geo_boundary(sensors['hexagon_8'][0])]

In [250]:
boundaries = {}

In [251]:
a = pd.DataFrame(boundari[0])
boundaries['max_lat'] = a[0].max()
boundaries['min_lat'] = a[0].min()
boundaries['max_lon'] = a[1].max()
boundaries['min_lon'] = a[1].min()

In [237]:
hex_coor_to_athena_geo(boundaries)

'polygon ((38.49158935182575 -121.44559247875796, 38.49509582934222 -121.44102691241622, 38.499783218066796 -121.44244926281223, 38.50096395450605 -121.44843753831455, 38.49745735834504 -121.45300291024557, 38.492770144400396 -121.45158020113382))'

In [254]:
print(f"""WHERE lat < {boundaries['max_lat']}
  AND lat > {boundaries['min_lat']}
  AND lon < {boundaries['max_lon']}
  AND lon > {boundaries['min_lon']}"""

WHERE lat < 38.50096395450605
  AND lat > 38.49158935182575
  AND lon < -121.44102691241622
  AND lon > -121.45300291024557
